In [1]:
import pandas as pd
from zipfile import ZipFile

with ZipFile("/user/wx2309/Topic-modeling-store/Processed data/past one year/contem_2023.csv.zip","r") as zip_ref:
    with zip_ref.open("contem_2023.csv") as file:
        contem_2023 = pd.read_csv(file)

In [2]:
contem_2023.head()

,date,rp_entity_id,comnam,ret,headline
0,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Business Disruptions Wane As Some Industries S...
1,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Press Release: KFC(R) Offers A Massive Deal on...
2,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Business News: Pizza Chains Race to Hire Driv...
3,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,KFC(R) Offers A Massive Deal on a Comfort Food...
4,2023-01-03,B4C673,YUM BRANDS INC,-0.008745,Business Disruptions Wane as Some Industries S...


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

headlines = contem_2023.headline.to_list()
vectorizer = CountVectorizer(max_df = 0.9)
doc_term = vectorizer.fit_transform(headlines)

In [8]:
from lda import LDA
ldamodel = LDA(n_topics = 62, n_iter = 100, random_state = 1)
ldamodel.fit(doc_term)

INFO:lda:n_documents: 1749770
INFO:lda:vocab_size: 153936
INFO:lda:n_words: 21183165
INFO:lda:n_topics: 62
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -274328302
INFO:lda:<10> log likelihood: -206912279
INFO:lda:<20> log likelihood: -177312229
INFO:lda:<30> log likelihood: -171354185
INFO:lda:<40> log likelihood: -169156334
INFO:lda:<50> log likelihood: -168081852
INFO:lda:<60> log likelihood: -167339891
INFO:lda:<70> log likelihood: -166903464
INFO:lda:<80> log likelihood: -166562071
INFO:lda:<90> log likelihood: -166333195
INFO:lda:<99> log likelihood: -166160262


In [9]:
import numpy as np

topic_dist = ldamodel.doc_topic_

contem_2023_topic_dist = pd.concat([contem_2023.drop(columns = ["rp_entity_id","headline"]),\
                                    pd.DataFrame(topic_dist)], axis =1)
grouped_sum = contem_2023_topic_dist.groupby(["date","comnam","ret"]).sum()

X = np.array(grouped_sum)
ret = [ind[2] for ind in list(grouped_sum.index)]
Y = np.array(ret).reshape(-1,1)

from sklearn.linear_model import LinearRegression
lda_model = LinearRegression(fit_intercept=True)
lda_model.fit(X,Y)
lda_model.score(X,Y)

0.01301239792127229

In [12]:
import pickle

with open("/user/wx2309/Topic-modeling-store/LDA_sp500_2023_contem_model_folder/LDA_sp500_2023_contem_model.pkl", "wb") as file:
    pickle.dump(ldamodel,file)